<a href="https://colab.research.google.com/github/devreborges/projetos_analise_dados/blob/main/programa_dev_reborges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
from urllib.request import urlopen, Request
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
import pandas as pd


In [122]:
df_linhas = pd.read_excel('./WO_WIPO_Marcas.xlsx')

In [123]:
df_linhas['NUMERO'].head()

0    1701130
1    1705961
2    1716118
3    1697722
4    1699766
Name: NUMERO, dtype: object

In [124]:
df_linhas['NUMERO'].isnull().value_counts()

NUMERO
False    10522
Name: count, dtype: int64

In [125]:
df_linhas['NUMERO'] = df_linhas['NUMERO'].apply(str)


In [126]:
df_linhas['NUMERO'].head()

0    1701130
1    1705961
2    1716118
3    1697722
4    1699766
Name: NUMERO, dtype: object

In [127]:
lista = df_linhas['NUMERO']

dados = []

In [ ]:
for i in lista:
    url = f"https://www3.wipo.int/madrid/monitor/en/showData.jsp?ID={i}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36"
    }

    req = Request(url, headers=headers)

    try:
        response = urlopen(req)
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        # Encontrando a div com a classe "lapin client repType"
        div_lapin = soup.find('div', class_='lapin client repType')

        if div_lapin is not None:
            # Extraindo o texto da div
            mandatario = div_lapin.get_text(separator='\n').strip()
            dados.append(mandatario)
        else:
            dados.append('FORA DO PADRAO HTML')  # Adicionando a mensagem caso não haja dados disponíveis

    except HTTPError as e:
        print('HTTP Error:', e.code, url)
        dados.append('FORA DO PADRAO HTML')  # Adicionando a mensagem caso ocorra um erro HTTP
    except URLError as e:
        print('URL Error:', e.reason, url)
        dados.append('FORA DO PADRAO HTML')  # Adicionando a mensagem caso ocorra um erro de URL

data_dict = {
    'NUMERO': lista,
    'MANDATARIO': dados
}


In [ ]:
# Criando o DataFrame do Pandas
df = pd.DataFrame(data_dict)


In [ ]:
df.to_excel('WO_WIPO_completo.xlsx', index=False)
